## Multi-threaded Parallelism and GPU Support

author: Jacob Schreiber <br>
contact: jmschreiber91@gmail.com

pomegranate has built in parallism based off of joblib. Because pomegranate is written almost entirely in Cython, the GIL can be released, allowing multithreading to be used instead of multiprocessing. Multithreading can be significantly more efficient because it is shared memory, meaning that all of the threads can operate on the same block of memory. In contrast, multiprocessing is not shared memory, and so the data has to be copied across multiple processes in ordr to be utilized.

These functions can all be simply parallelized by passing in `n_jobs=X` to the method calls, both fitting and inference. This tutorial will demonstrate how to use those calls. First we'll look at a simple multivariate Gaussian mixture model, and compare its performance to sklearn. Then we'll look at a hidden Markov model with Gaussian emissions, and lastly we'll look at a mixture of Gaussian HMMs. These can all utilize the build-in parallelization that pomegranate has.

Let's dive right in!

In [1]:
%matplotlib inline
import time
import pandas
import random
import numpy
import matplotlib.pyplot as plt
import seaborn; seaborn.set_style('whitegrid')
import itertools

from pomegranate import *

random.seed(0)
numpy.random.seed(0)
numpy.set_printoptions(suppress=True)

%load_ext watermark
%watermark -m -n -p numpy,scipy,pomegranate

Sat Nov 03 2018 

numpy 1.14.2
scipy 1.0.0
pomegranate 0.10.0

compiler   : GCC 7.2.0
system     : Linux
release    : 4.15.0-36-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit


In [2]:
from sklearn.mixture import GaussianMixture

def create_dataset(n_samples, n_dim, n_classes, alpha=1):
    """Create a random dataset with n_samples in each class."""
    
    X = numpy.concatenate([numpy.random.normal(i*alpha, 1, size=(n_samples, n_dim)) for i in range(n_classes)])
    y = numpy.concatenate([numpy.zeros(n_samples) + i for i in range(n_classes)])
    idx = numpy.arange(X.shape[0])
    numpy.random.shuffle(idx)
    return X[idx], y[idx]

### 1. Naive Bayes models

Let's start off with one of the simplest models, the naive Bayes model. This model is simple because it models each feature independently of the others.

In [3]:
from sklearn.naive_bayes import GaussianNB

X, y = create_dataset(10000, 5000, 2)

%timeit NaiveBayes.from_samples(NormalDistribution, X, y)
%timeit NaiveBayes.from_samples(NormalDistribution, X, y, n_jobs=4)
%timeit GaussianNB().fit(X, y)

KeyboardInterrupt: 

We can also make predictions using the model in parallel.

In [ ]:
model = NaiveBayes.from_samples(NormalDistribution, X, y)
model2 = GaussianNB().fit(X, y)

%timeit model.predict_proba(X)
%timeit model.predict_proba(X, n_jobs=4)
%timeit model2.predict_proba(X)

We can check to make sure that the predictions made when using parallelism aren't different than those done without parallelism for some reason.

In [ ]:
(model.predict_proba(X) - model.predict_proba(X, n_jobs=4)).sum()

Great!

### 2. Mixture Models

Let's now look at training mitures models, which require the more time-intensive EM algorithm to train.

In [ ]:
from sklearn.mixture import GaussianMixture

X = numpy.concatenate([numpy.random.normal(0, 3, size=(20000, 500)),
                       numpy.random.normal(1, 3, size=(10000, 500))])

%timeit GeneralMixtureModel.from_samples(NormalDistribution, 2, X, max_iterations=10)
%timeit GeneralMixtureModel.from_samples(NormalDistribution, 2, X, max_iterations=10, n_jobs=4)
%timeit GaussianMixture(2, covariance_type='diag', max_iter=10).fit(X)

We can also check the time that it takes to perform inference.

In [ ]:
model = GeneralMixtureModel.from_samples(NormalDistribution, 2, X, max_iterations=10)
model2 = GaussianMixture(2, covariance_type='diag', max_iter=10).fit(X)

%timeit model.predict_proba(X)
%timeit model.predict_proba(X, n_jobs=4)
%timeit model2.predict_proba(X)

We can check again that there is no difference in the predictions.

In [ ]:
(model.predict_proba(X) - model.predict_proba(X, n_jobs=4)).sum()

### 3. Hiden Markov Models

Now let's move on to training a hidden Markov model with multivariate Gaussian emissions with a diagonal covariance matrix. We'll randomly generate some Gaussian distributed numbers and use pomegranate with either one or four threads to fit our model to the data.

In [ ]:
X = numpy.random.randn(1000, 500, 50)

print "pomegranate Gaussian HMM (1 job)"
%timeit -n 1 -r 1 HiddenMarkovModel.from_samples(NormalDistribution, 5, X, max_iterations=5)
print
print "pomegranate Gaussian HMM (2 jobs)"
%timeit -n 1 -r 1 HiddenMarkovModel.from_samples(NormalDistribution, 5, X, max_iterations=5, n_jobs=2)
print
print "pomegranate Gaussian HMM (2 jobs)"
%timeit -n 1 -r 1 HiddenMarkovModel.from_samples(NormalDistribution, 5, X, max_iterations=5, n_jobs=4)

All we had to do was pass in the n_jobs parameter to the fit function in order to get a speed improvement. It looks like we're getting a really good speed improvement, as well! This is mostly because the HMM algorithms perform a lot more operations than the other models, and so spend the vast majority of time with the GIL released. You may not notice as strong speedups when using a MultivariateGaussianDistribution because BLAS uses multithreaded operations already internally, even when only one job is specified.

Now lets look at the prediction function to make sure the we're getting speedups there as well. You'll have to use a wrapper function to parallelize the predictions for a HMM because it returns an annotated sequence rather than a single value like a classic machine learning model might.

In [ ]:
model = HiddenMarkovModel.from_samples(NormalDistribution, 5, X, max_iterations=2, verbose=False)

print "pomegranate Gaussian HMM (1 job)"
%timeit predict_proba(model, X)
print
print "pomegranate Gaussian HMM (2 jobs)"
%timeit predict_proba(model, X, n_jobs=2)

Great, we're getting a really good speedup on that as well! Looks like the parallel processing is more efficient with a bigger, more complex model, than with a simple one. This can make sense, because all inference/training is more complex, and so there is more time with the GIL released compared to with the simpler operations.

### 4. Mixture of Hidden Markov Models

Let's stack another layer onto this model by making it a mixture of these hidden Markov models, instead of a single one. At this point we're sticking a multivariate Gaussian HMM into a mixture and we're going to train this big thing in parallel.

In [ ]:
def create_model(mus):
    n = mus.shape[0]
    
    starts = numpy.zeros(n)
    starts[0] = 1.
    
    ends = numpy.zeros(n)
    ends[-1] = 0.5
    
    transition_matrix = numpy.zeros((n, n))
    distributions = []
    
    for i in range(n):
        transition_matrix[i, i] = 0.5
        
        if i < n - 1:
            transition_matrix[i, i+1] = 0.5
    
        distribution = IndependentComponentsDistribution([NormalDistribution(mu, 1) for mu in mus[i]])
        distributions.append(distribution)
    
    model = HiddenMarkovModel.from_matrix(transition_matrix, distributions, starts, ends)
    return model
    

def create_mixture(mus):
    hmms = [create_model(mu) for mu in mus]
    return GeneralMixtureModel(hmms)

n, d = 50, 10
mus = [(numpy.random.randn(d, n)*0.2 + numpy.random.randn(n)*2).T for i in range(2)]

In [ ]:
model = create_mixture(mus)
X = numpy.random.randn(400, 150, d)

print "pomegranate Mixture of Gaussian HMMs (1 job)"
%timeit model.fit(X, max_iterations=5)
print

model = create_mixture(mus)
print "pomegranate Mixture of Gaussian HMMs (2 jobs)"
%timeit model.fit(X, max_iterations=5, n_jobs=2)

Looks like we're getting a really nice speed improvement when training this complex model. Let's take a look now at the time it takes to do inference with it.

In [ ]:
model = create_mixture(mus)

print "pomegranate Mixture of Gaussian HMMs (1 job)"
%timeit model.predict_proba(X)
print

model = create_mixture(mus)
print "pomegranate Mixture of Gaussian HMMs (2 jobs)"
%timeit model.predict_proba(X, n_jobs=2)

We're getting a good speed improvement here too through parallelization.

### 5. Bayesian Networks

In [ ]:
X = numpy.random.randint(2, size=(500, 18)).astype('float64')

%timeit BayesianNetwork.from_samples(X)
%timeit BayesianNetwork.from_samples(X, n_jobs=4)

Now let's check the speed that it takes to make predictions.

In [ ]:
i = numpy.random.randint(X.shape[0], size=500)
j = numpy.random.randint(X.shape[1], size=500)
X[i, j] = numpy.nan

model = BayesianNetwork.from_samples(X)

%timeit model.predict_proba(X)
%timeit model.predict_proba(X, n_jobs=4)

In [ ]:
model.predict_proba(X, n_jobs=4)[0]

In [ ]:
model.predict_proba(X)[0]

In [ ]:
print model.predict_proba(X[0])

model2 = BayesianNetwork.from_json(model.to_json())
model.states[0]

In [ ]:
model.to_json()

## Conclusions

Hopefully you'll find pomegranate useful in your work! Parallelization should allow you to train complex models faster than before. Keep in mind though that there is an overhead to using parallel processing, and so it's possible that on some smaller examples it does not work as well. In general, the bigger the dataset, the closer to a linear speedup you'll get with pomegranate.

If you have any interesting examples of how you've used pomegranate in your work, I'd love to hear about them. In addition I'd like to hear any feedback you may have on features you'd like to see. Please shoot me an email. Good luck!